### Import thư viện

In [1]:
import numpy as np                  #tính toán ma trận
import matplotlib.pyplot as plt     #hiển thị ảnh
from PIL import Image               #đọc, ghi ảnh

### Nhóm hàm chức năng

In [2]:
def change_brightness(img):
    return [np.clip(img+50, 0, 255), np.clip(img-50, 0, 255)]


def change_contrast(img):
    return [np.clip(img*1.5, 0, 255), np.clip(img*0.5, 0, 255)]


def flip_image(img, direction=''):
    if direction=='horizontal':    #horizontal -   dọc  - left to right
        return [img[:,::-1]]
    elif direction=='vertical':  #vertical - ngang
        return [img[::-1,:]]
    return [img[:,::-1], img[::-1,:]]


def convert_to_grayscale(img):
    return np.clip(np.dot(img[...,:3], [0.2989, 0.5870, 0.1140]), 0, 255)


def convert_to_sepia(img):
    tr = np.dot(img[...,:3], [0.393, 0.769, 0.189]) 
    tg = np.dot(img[...,:3], [0.349, 0.686, 0.168]) 
    tb = np.dot(img[...,:3], [0.272, 0.534, 0.131])

    new_img = np.zeros_like(img)
    new_img[:,:,0] = tr
    new_img[:,:,1] = tg
    new_img[:,:,2] = tb

    return np.clip(new_img, 0, 255) 


def grayscale_and_sepia(img):
    return [convert_to_grayscale(img), convert_to_sepia(img)]


def blur_and_sharpen(img):
    kernel = np.array([[1.0, 2.0, 1.0],
                       [2.0, 4.0, 2.0],
                       [1.0, 2.0, 1.0]]
                    )
    kernel = kernel/np.sum(kernel)

    arraylist = []
    for y in range(3):
        temparray = np.copy(img)
        temparray = np.roll(temparray, y - 1, axis=0)
        for x in range(3):
            temparray_X = np.copy(temparray)
            temparray_X = np.roll(temparray_X, x - 1, axis=1)*kernel[y,x]
            arraylist.append(temparray_X)

    arraylist = np.array(arraylist)
    blur_img = np.sum(arraylist, axis=0)
    return [blur_img, np.clip(2*img - blur_img, 0, 255)]

def center_crop(img):
    x,y,_ = img.shape
    return [img[x//4 : x-x//4 , y//4 : y - y//4, :]]

def circle_mask(img):
    a, b, _ = img.shape
    X, Y = np.ogrid[0:a, 0:b]

    R = a/2
    if a>b:
        R = b/2

    mask = (X - a/2) ** 2 + (Y - b/2)**2 > R**2

    new_img = img.copy()
    new_img[mask] = 0

    return [new_img]

def ellipse_mask(img):
    # define ellipse parameters
    distance = img.shape[1]/2
    a = distance*(np.sqrt(2) - 0.2) # major axis
    b = 3*a/5  # minor axis
    theta = np.arctan(img.shape[1]/img.shape[0])  # angle of rotation in radians

    # create meshgrid
    x_c, y_c = img.shape[1] // 2, img.shape[0] // 2
    h, w = img.shape[:2]
    y, x = np.ogrid[-y_c:h-y_c, -x_c:w-x_c]

    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    ellipse_1 = (((x * cos_theta - y * sin_theta) ** 2) / (a ** 2)) + (((x * sin_theta + y * cos_theta) ** 2) / (b ** 2)) <= 1
    ellipse_2 = (((x * np.cos(theta + np.pi / 2) + y * np.sin(theta + np.pi / 2)) ** 2) / (b ** 2)) + (((-x * np.sin(theta + np.pi / 2) + y * np.cos(theta + np.pi / 2)) ** 2) / (a ** 2)) <= 1

    ellipse_mask = ellipse_1 + ellipse_2

    cropped_ellipse_img = np.zeros_like(img)
    for c in range(img.shape[-1]):
        cropped_ellipse_img[:, :, c] = img[:, :, c] * ellipse_mask
    
    return [cropped_ellipse_img]

### Khai báo danh sách lưu trữ các hàm chức năng & tên hàm chức năng

In [3]:
func_name = [['brighter', 'darker'],
             ['higher_contrast', 'lower_contrast'],
             ['horizontal_flip', 'vertical_flip'],
             ['grayscale', 'sepia'],
             ['blur', 'sharpen'],
             ['center_crop'],
             ['circle_mask'],
             ['ellipse_mask']
            ]

function = [change_brightness,
            change_contrast,
            flip_image,
            grayscale_and_sepia,
            blur_and_sharpen,
            center_crop,
            circle_mask,
            ellipse_mask
            ]

direction = ['horizontal',
             'vertical']

### Nhóm hàm lấy dữ liệu (từ người dùng, từ ảnh, ...)

In [4]:
def input_from_user():
    filename = input('filename: ')

    while True:
        try:
            func_id = int(input('function: '))
            assert 0 <= func_id <= 8
        except ValueError:
            print('Not an integer! Please enter an integer.')
        except AssertionError:
            print('Value of input must be in range of [0,7].')
        else:
            break
    
    if func_id == 3:
        while True:
            try:
                dir_id = int(input('type of flipping [0:horizontal/  1:vertical]')) #horizontal là vd của cô
                assert 0 <= dir_id <= 1
            except ValueError:
                print('Not an integer! Please enter an integer.')
            except AssertionError:
                print('Value of input must be in range of [0,1].')
            else:
                return filename, func_id, direction[dir_id]
    return filename, func_id, 'both'
    
def data_from_img(filename):
    img = Image.open(filename)
    return np.array(img, dtype=float)

### Nhóm hàm xuất dữ liệu (ra console, ra ảnh, ...)

In [5]:
def save_img(img_list, filein, func_id, direction='both'):
    index = filein.rfind('.')
    img_name = filein[ : index]
    img_format = filein[index : ]
    
    for _ in range(len(img_list)):
        if direction!='both':
            fileout = img_name + '_' + func_name[func_id][direction] + img_format
        else:
            fileout = img_name + '_' + func_name[func_id][_] + img_format

        if func_id==3 and _==0:
            plt.imsave(f"{fileout}", img_list[_].astype(np.uint8), cmap='gray')
        else:
            plt.imsave(f"{fileout}", img_list[_].astype(np.uint8))

### main

In [6]:
from timeit import default_timer as timer

def program(filename, func_id, direction):
    img= data_from_img(filename)

    if (func_id==0):
        for _ in range(8):
            func = function[_]

            start = timer()
            img_list = func(img)
            end = timer()
            print('runtime ', func_name[_], ': ', end-start)
                
            save_img(img_list, filename, _)
        return

    func_id-=1
    func = function[func_id]

    if func==flip_image:
        img_list = func(img, direction)
        save_img(img_list, filename, func_id, direction)
    else:
        img_list = func(img)
        save_img(img_list, filename, func_id)

In [7]:
def main():
    filename, func_id, direction  = input_from_user()
    program(filename, func_id, direction)        

if __name__ == "__main__":
    main()

runtime  ['brighter', 'darker'] :  0.009773800004040822
runtime  ['higher_contrast', 'lower_contrast'] :  0.013199699998949654
runtime  ['horizontal_flip', 'vertical_flip'] :  0.0010291999933542684
runtime  ['grayscale', 'sepia'] :  0.03407359999255277
runtime  ['blur', 'sharpen'] :  0.16040399999474175
runtime  ['center_crop'] :  0.0011823000095319003
runtime  ['circle_mask'] :  0.004030900003272109
runtime  ['eclipse_mask'] :  0.02055270000710152
